In [1]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import re
import string
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import pymorphy2
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/iromaykin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/iromaykin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/iromaykin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/iromaykin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Предобработка текста

In [2]:
class TextPreprocessor:
    def __init__(self, method='lemmatization', tokenizer='word', language='russian'):
        """
        Инициализация объекта класса предобработки текста.
        
        Параметры:
            method (str): Метод обработки слов - 'stemming' или 'lemmatization'.
            tokenizer (str): Тип токенизации - 'letter', 'word' или 'sentence'.
            language (str): Язык текстов - 'russian' или 'english'
        """
        self.method = method
        self.tokenizer = tokenizer
        self.language = language
        self.stop_words = set(stopwords.words(self.language))
        self.stemmer = SnowballStemmer(self.language)
        self.lemmatizer_ru = pymorphy2.MorphAnalyzer()
        self.lemmatizer_en = WordNetLemmatizer()
    
    def get_wordnet_pos(self, word):
        """Функция для преобразования POS-тегов из NLTK в формат, подходящий для WordNetLemmatizer"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {
            'J': wordnet.ADJ,
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'R': wordnet.ADV
        }
        return tag_dict.get(tag, wordnet.NOUN)
    
    def clean_text(self, text):
        """Удаляет спецсимволы, HTML-теги, ссылки и номера."""
        text = BeautifulSoup(text, "html.parser").get_text() # Удаляем HTML
        text = re.sub(r'http\S+', '', text)  # Удаление ссылок
        text = re.sub(r'\d+', '', text)  # Удаление номеров
        if self.tokenizer == 'sentence':
            text = re.sub('[^a-zA-Zа-яА-Я0-9.\s]', '', text)  # Удаление пунктуации (кроме точки, т.к. она важна для разделения предложений
            text = re.sub('\.', ' . ', text)  # Разъединяем точку с словами
        else:
            text = re.sub('[^a-zA-Zа-яА-Я0-9\s]', '', text)  # Удаление пунктуации
        text = re.sub(r'\s+', ' ', text) # удаление лишних пробелов
        return text
    
    def to_lowercase(self, text):
        """Приводит текст к нижнему регистру."""
        return text.lower()
    
    def remove_stopwords(self, text):
        """Удаляет стоп-слова."""
        return ' '.join([word for word in text.split(' ') if word not in self.stop_words])
    
    def process_word(self, text):
        """Выполняет стемминг или лемматизацию слова в зависимости от выбранного метода."""
        text = text.split(' ')
        if self.method == 'stemming':
            result = [self.stemmer.stem(word) for word in text]
        elif self.method == 'lemmatization' and self.language == 'russian':
            result = [self.lemmatizer_ru.parse(word)[0].normal_form for word in text]
        elif self.method == 'lemmatization' and self.language == 'english':
            result = [self.lemmatizer_en.lemmatize(word, self.get_wordnet_pos(word)) for word in text]

        return ' '.join(result)
    
    def tokenize(self, text):
        """Выполняет токенизацию текста."""
        if self.tokenizer == 'letter':
            return list(text)
        elif self.tokenizer == 'word':
            return word_tokenize(text, language='russian')
        elif self.tokenizer == 'sentence':
            return sent_tokenize(text, language='russian')
    
    def preprocess_texts(self, texts):
        """Выполняет предобработку списка текстов."""
        processed_texts = []
        for text in texts:
            cleaned_text = self.clean_text(text)
            lowercase_text = self.to_lowercase(cleaned_text)
            text_without_stopwords = self.remove_stopwords(lowercase_text)
            process_text = self.process_word(text_without_stopwords)
            tokens = self.tokenize(process_text)
            processed_texts.append(tokens)
        
        return processed_texts

In [3]:
text = 'Пример текста для <b>предобработки</b>. Этот текст содержит ссылку http://example.com и номера 12345. \
        NLP — это область искусственного интеллекта, связанная с обработкой и анализом естественного языка. \
        Она включает в себя множество задач, таких как машинный перевод, анализ тональности, извлечение информации и многие другие.'

preprocessor = TextPreprocessor(method='lemmatization', tokenizer='sentence', language='russian')
print(' Default text: ')
print(text)
clean_text = preprocessor.clean_text(text)
print('\n clean text: ')
print(clean_text)
lower_text = preprocessor.to_lowercase(clean_text)
print('\n lower_text: ')
print(lower_text)
text_without_stopwords = preprocessor.remove_stopwords(lower_text)
print('\n text_without_stopwords: ')
print(text_without_stopwords)
lematize_text = preprocessor.process_word(text_without_stopwords)
print('\n lem_text: ')
print(lematize_text)
tokens = preprocessor.tokenize(lematize_text)
print('\n tokens: ')
print(tokens)

 Default text: 
Пример текста для <b>предобработки</b>. Этот текст содержит ссылку http://example.com и номера 12345.         NLP — это область искусственного интеллекта, связанная с обработкой и анализом естественного языка.         Она включает в себя множество задач, таких как машинный перевод, анализ тональности, извлечение информации и многие другие.

 clean text: 
Пример текста для предобработки . Этот текст содержит ссылку и номера . NLP это область искусственного интеллекта связанная с обработкой и анализом естественного языка . Она включает в себя множество задач таких как машинный перевод анализ тональности извлечение информации и многие другие . 

 lower_text: 
пример текста для предобработки . этот текст содержит ссылку и номера . nlp это область искусственного интеллекта связанная с обработкой и анализом естественного языка . она включает в себя множество задач таких как машинный перевод анализ тональности извлечение информации и многие другие . 

 text_without_stopwords: 

In [4]:
print('Stemming:')
preprocessor = TextPreprocessor(method='stemming', tokenizer='word', language='russian')
processed_texts = preprocessor.preprocess_texts([text])
for result in processed_texts:
    print(result)
    print('\n')

print('Lemmatization:')
preprocessor = TextPreprocessor(method='lemmatization', tokenizer='word', language='russian')
processed_texts = preprocessor.preprocess_texts([text])
for result in processed_texts:
    print(result)
    print('\n')

Stemming:
['пример', 'текст', 'предобработк', 'текст', 'содерж', 'ссылк', 'номер', 'nlp', 'эт', 'област', 'искусствен', 'интеллект', 'связа', 'обработк', 'анализ', 'естествен', 'язык', 'включа', 'множеств', 'задач', 'так', 'машин', 'перевод', 'анализ', 'тональн', 'извлечен', 'информац', 'мног', 'друг']


Lemmatization:
['пример', 'текст', 'предобработка', 'текст', 'содержать', 'ссылка', 'номер', 'nlp', 'это', 'область', 'искусственный', 'интеллект', 'связать', 'обработка', 'анализ', 'естественный', 'язык', 'включать', 'множество', 'задача', 'такой', 'машинный', 'перевод', 'анализ', 'тональность', 'извлечение', 'информация', 'многие', 'другой']




# Тестирование влияния предобработки текста

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [6]:
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "split")
ds

/opt/conda/envs/interns_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [16]:
df_train = ds['train'].to_pandas().sample(4000)
display(df_train)
df_test = ds['test'].to_pandas()
display(df_test)

,text,label
446,i feel very comfortable with this decision,1
6103,i love that i feel valuable i love making the ...,1
7973,i could loose my job i would be so f amp ed fo...,4
4730,i feel popular but they dont want to be taught...,1
1586,im feeling adventurous and my laundry hamper,1
...,...,...
96,im starting to feel wryly amused at the banal ...,1
15676,i feel so much better about that number,1
11708,i feel absolutely overwhelmed by it,4
10285,i went to pick up the kids feeling scared and ...,4


,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [17]:
preprocessor = TextPreprocessor(method='lemmatization', tokenizer='word', language='english')
df_train['preprocessed_text'] = preprocessor.preprocess_texts(df_train['text'])
df_train['preprocessed_text'] = df_train['preprocessed_text'].apply(lambda x: ' '.join(x))
display(df_train)
df_test['preprocessed_text'] = preprocessor.preprocess_texts(df_test['text'])
df_test['preprocessed_text'] = df_test['preprocessed_text'].apply(lambda x: ' '.join(x))
display(df_test)

,text,label,preprocessed_text
446,i feel very comfortable with this decision,1,feel comfortable decision
6103,i love that i feel valuable i love making the ...,1,love feel valuable love make choice love easy ...
7973,i could loose my job i would be so f amp ed fo...,4,could loose job would f amp ed xmas hate xmas ...
4730,i feel popular but they dont want to be taught...,1,feel popular dont want taught wont get married...
1586,im feeling adventurous and my laundry hamper,1,im feel adventurous laundry hamper
...,...,...,...
96,im starting to feel wryly amused at the banal ...,1,im start feel wryly amuse banal comedy error l...
15676,i feel so much better about that number,1,feel much well number
11708,i feel absolutely overwhelmed by it,4,feel absolutely overwhelmed
10285,i went to pick up the kids feeling scared and ...,4,go pick kid feel scar trembly self critical st...


,text,label,preprocessed_text
0,im feeling rather rotten so im not very ambiti...,0,im feel rather rotten im ambitious right
1,im updating my blog because i feel shitty,0,im update blog feel shitty
2,i never make her separate from me because i do...,0,never make separate ever want feel like ashamed
3,i left with my bouquet of red and yellow tulip...,1,left bouquet red yellow tulip arm feel slightl...
4,i was feeling a little vain when i did this one,0,feel little vain one
...,...,...,...
1995,i just keep feeling like someone is being unki...,3,keep feel like someone unkind wrong think get ...
1996,im feeling a little cranky negative after this...,3,im feel little cranky negative doctor appointment
1997,i feel that i am useful to my people and that ...,1,feel useful people give great feel achievement
1998,im feeling more comfortable with derby i feel ...,1,im feel comfortable derby feel though start st...


In [18]:
# Векторизация текстов с использованием TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, lowercase=False, stop_words=None, analyzer='word')
X_train = vectorizer.fit_transform(df_train['preprocessed_text'])
X_test = vectorizer.transform(df_test['preprocessed_text'])


vectorizer_original_texts = TfidfVectorizer(max_features=1000, lowercase=False, stop_words=None, analyzer='word')
X_train_original_texts = vectorizer.fit_transform(df_train['text'])
X_test_original_texts = vectorizer.transform(df_test['text'])

y_train, y_test = df_train['label'], df_test['label']

In [25]:
# Обучение модели линейной регрессии на предобработанных текстах
LR_model_1 = LogisticRegression(max_iter=1000)
LR_model_1.fit(X_train, y_train)
predictions = LR_model_1.predict(X_test)
f1 = f1_score(y_test, predictions, average='macro')
print(f'F1 (Logistic Regression on Preprocessed Texts): {f1}')

# Обучение модели линейной регрессии на непредобработанных текстах
LR_model_2 = LogisticRegression(max_iter=1000)
LR_model_2.fit(X_train_original_texts, y_train)
predictions_processed = LR_model_2.predict(X_test_original_texts)
f1_processed = f1_score(y_test, predictions_processed, average='macro')
print(f'F1 (Logistic Regression on Original Texts): {f1_processed}')

F1 (Logistic Regression on Preprocessed Texts): 0.6767943456685482
F1 (Logistic Regression on Original Texts): 0.6267488913124267


In [24]:
from sklearn.ensemble import RandomForestClassifier

# Обучение модели случайного леса на предобработанных текстах
RF_model_1 = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=42)
RF_model_1.fit(X_train, y_train)
predictions_rf = RF_model_1.predict(X_test)
f1_rf = f1_score(y_test, predictions_rf, average='macro')
print(f'F1 (Random Forest on Preprocessed Texts): {f1_rf}')

# Обучение модели случайного леса на непредобработанных текстах
RF_model_2 = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=42)
RF_model_2.fit(X_train_original_texts, y_train)
predictions_rf_processed = RF_model_2.predict(X_test_original_texts)
f1_rf_processed = f1_score(y_test, predictions_rf_processed, average='macro')
print(f'F1 (Random Forest on Original Texts): {f1_rf_processed}')


F1 (Random Forest on Preprocessed Texts): 0.595787765277807
F1 (Random Forest on Original Texts): 0.622330943974163


In [22]:
from sklearn.neural_network import MLPClassifier

# Обучение MLP на предобработанных текстах
MLP_model_1 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
MLP_model_1.fit(X_train, y_train)
predictions_mlp = MLP_model_1.predict(X_test)
f1_mlp = f1_score(y_test, predictions_mlp, average='macro')
print(f'F1 (MLP on Preprocessed Texts): {f1_mlp}')

# Обучение MLP на непредобработанных текстах
MLP_model_2 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
MLP_model_2.fit(X_train_original_texts, y_train)
predictions_mlp_processed = MLP_model_2.predict(X_test_original_texts)
f1_mlp_processed = f1_score(y_test, predictions_mlp_processed, average='macro')
print(f'F1 (MLP on Original Texts): {f1_mlp_processed}')


F1 (MLP on Preprocessed Texts): 0.7038576786240373
F1 (MLP on Original Texts): 0.6835235961994314


# Рекомендации по предобработке текста (выводы)

### 1. Ограниченная предобработка текста

- Не всегда полная предобработка текста способствует улучшению показателей модели.
- В некоторых случаях достаточно ограничиться удалением HTML-тегов, ссылок и приведением текста к нижнему регистру, чтобы добиться хороших результатов.

### 2. Влияние модели на результаты предобработки

- Разные модели могут по-разному реагировать на предобработанные и непредобработанные тексты.
- Важно тестировать несколько моделей, чтобы определить, какая из них лучше справляется с данными в зависимости от степени предобработки.

### 3. Важность полной предобработки при малом объеме данных

- При небольшом объеме данных предобработка становится критически важной, так как множество различных способов написания одних и тех же слов без лемматизации могут быть интерпретированы как разные слова.*
- Лемматизация помогает модели правильно сопоставлять слова, увеличивая её способность обобщать и правильно классифицировать тексты.

### 4. Важность полной предобработки при большом объеме данных и ограниченных ресурсах

- При больших объемах данных и ограниченных вычислительных ресурсах, предобработка может помочь уменьшить размерность данных и ускорить обучение модели.
- Полная предобработка уменьшает количество уникальных слов, что снижает размерность TF-IDF векторов и сокращает время обучения модели.

## Заключение

Подход к предобработке текста должен быть адаптирован под конкретные условия использования, включая тип модели, размер и сложность данных, а также доступные вычислительные ресурсы. Регулярное тестирование и оценка различных подходов к предобработке помогут оптимизировать процесс и улучшить результаты классификации.


****К примеру, у нас 200 строк на 1 класс. У этого класса есть слово триггер - "get", у которого формы: get, gotten(have got), gets, got, getting (т.е. когда оно есть, то мы с большой вероятностью можем определить объект к этому классу); Для ML-модели все эти формы будут разными словами (это будут разные токены), а если еще учесть что это слово будет встречается не во всех строках, то определить его без лемматизации будет достаточно сложно.***